In [1]:
from mtr.datasets import build_dataloader

In [2]:
from mtr.config import cfg, cfg_from_yaml_file
from mtr.utils import common_utils
import torch

In [3]:
cfg_from_yaml_file("/code/jjiang23/csc587/KimchiVision/cfg/kimchiConfig.yaml", cfg)

{'ROOT_DIR': PosixPath('/code/jjiang23/csc587/KimchiVision'),
 'LOCAL_RANK': 0,
 'DATA_CONFIG': {'DATASET': 'WaymoDataset',
  'OBJECT_TYPE': ['TYPE_VEHICLE', 'TYPE_PEDESTRIAN', 'TYPE_CYCLIST'],
  'DATA_ROOT': '/files/waymo/code/MTR/data/waymo',
  'SPLIT_DIR': {'train': 'processed_scenarios_training',
   'test': 'processed_scenarios_validation'},
  'INFO_FILE': {'train': 'processed_scenarios_training_infos.pkl',
   'test': 'processed_scenarios_val_infos.pkl'},
  'SAMPLE_INTERVAL': {'train': 1, 'test': 1},
  'INFO_FILTER_DICT': {'filter_info_by_object_type': ['TYPE_VEHICLE',
    'TYPE_PEDESTRIAN',
    'TYPE_CYCLIST']},
  'POINT_SAMPLED_INTERVAL': 1,
  'NUM_POINTS_EACH_POLYLINE': 20,
  'VECTOR_BREAK_DIST_THRESH': 1.0,
  'NUM_OF_SRC_POLYLINES': 768,
  'CENTER_OFFSET_OF_MAP': [30.0, 0]},
 'MODEL': {'CONTEXT_ENCODER': {'NAME': 'MTREncoder',
   'NUM_OF_ATTN_NEIGHBORS': 16,
   'NUM_INPUT_ATTR_AGENT': 29,
   'NUM_INPUT_ATTR_MAP': 9,
   'NUM_CHANNEL_IN_MLP_AGENT': 256,
   'NUM_CHANNEL_IN_MLP_MAP

In [4]:
logger = common_utils.create_logger("/files/waymo/log.txt", rank=0)


In [5]:
from easydict import EasyDict as edict
args = edict({
    "batch_size": 2,
    "workers": 64,
    "merge_all_iters_to_one_epoch": False,
    "epochs": 10,
    "add_worker_init_fn": False,
})

In [6]:
train_set, train_loader, train_sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    batch_size=args.batch_size,
    dist=False, workers=args.workers,
    logger=logger,
    training=True,
    merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch,
    total_epochs=args.epochs,
    add_worker_init_fn=args.add_worker_init_fn,
)

2025-06-07 04:15:26,824   INFO  Start to load infos from /files/waymo/code/MTR/data/waymo/processed_scenarios_training_infos.pkl
2025-06-07 04:15:32,045   INFO  Total scenes before filters: 243401
2025-06-07 04:15:37,895   INFO  Total scenes after filter_info_by_object_type: 243401
2025-06-07 04:15:37,919   INFO  Total scenes after filters: 243401


In [7]:
sample = next(iter(train_loader))

In [8]:
sample.keys()

dict_keys(['batch_size', 'input_dict', 'batch_sample_count'])

In [9]:
sample["batch_size"]

2

In [10]:
input_dict = sample["input_dict"]

In [11]:
for key in input_dict.keys():
    if isinstance(input_dict[key], torch.Tensor):
        print(f"{key}: {input_dict[key].shape}")
    else:
        print(f"{key}: {type(input_dict[key])}")

scenario_id: <class 'numpy.ndarray'>
obj_trajs: torch.Size([8, 29, 11, 29])
obj_trajs_mask: torch.Size([8, 29, 11])
track_index_to_predict: torch.Size([8])
obj_trajs_pos: torch.Size([8, 29, 11, 3])
obj_trajs_last_pos: torch.Size([8, 29, 3])
obj_types: <class 'numpy.ndarray'>
obj_ids: <class 'numpy.ndarray'>
center_objects_world: torch.Size([8, 10])
center_objects_id: <class 'numpy.ndarray'>
center_objects_type: <class 'numpy.ndarray'>
obj_trajs_future_state: torch.Size([8, 29, 80, 4])
obj_trajs_future_mask: torch.Size([8, 29, 80])
center_gt_trajs: torch.Size([8, 80, 4])
center_gt_trajs_mask: torch.Size([8, 80])
center_gt_final_valid_idx: torch.Size([8])
center_gt_trajs_src: torch.Size([8, 91, 10])
map_polylines: torch.Size([8, 768, 20, 9])
map_polylines_mask: torch.Size([8, 768, 20])
map_polylines_center: torch.Size([8, 768, 3])
static_map_polylines: torch.Size([8, 4000, 20, 7])
static_map_polylines_mask: torch.Size([8, 4000, 20])


In [26]:
input_dict['map_polylines'][0]

tensor([[[ 33.2058, 106.4469,  -2.3288,  ...,  15.0000,  33.2058, 106.4469],
         [ 32.9673, 106.0101,  -2.3265,  ...,  15.0000,  33.2058, 106.4469],
         [ 32.7280, 105.5742,  -2.3242,  ...,  15.0000,  32.9673, 106.0101],
         ...,
         [ 29.1033,  99.0517,  -2.2893,  ...,  15.0000,  29.3422,  99.4883],
         [ 28.8659,  98.6148,  -2.2869,  ...,  15.0000,  29.1033,  99.0517],
         [ 28.6296,  98.1769,  -2.2846,  ...,  15.0000,  28.8659,  98.6148]],

        [[ 28.3947,  97.7382,  -2.2823,  ...,  15.0000,  28.3947,  97.7382],
         [ 28.1617,  97.2991,  -2.2800,  ...,  15.0000,  28.3947,  97.7382],
         [ 27.9301,  96.8587,  -2.2776,  ...,  15.0000,  28.1617,  97.2991],
         ...,
         [ 24.7574,  90.1254,  -2.2287,  ...,  15.0000,  24.9505,  90.5813],
         [ 24.5661,  89.6692,  -2.2236,  ...,  15.0000,  24.7574,  90.1254],
         [ 24.3762,  89.2123,  -2.2185,  ...,  15.0000,  24.5661,  89.6692]],

        [[ 24.1875,  88.7552,  -2.2134,  ...

In [ ]:
input_dict["static_map_polylines"].shape, input_dict["static_map_polylines_mask"].shape

In [ ]:
input_dict["static_map_polylines"].shape, input_dict["static_map_polylines_mask"].shape

In [ ]:
input_dict["map_polylines"].shape

In [ ]:
input_dict["center_objects_world"].shape

In [ ]:
input_dict["map_polylines_mask"].shape

In [ ]:
input_dict["obj_trajs_mask"][0][0]

In [ ]:
input_dict["obj_trajs"].shape

In [ ]:
# (num_center_objects, num_topk_polylines, num_points_each_polyline, 9): [x, y, z, dir_x, dir_y, dir_z, global_type, pre_x, pre_y]
sample["input_dict"]['map_polylines'].shape

In [ ]:
input_dict["map_polylines_center"].shape

In [ ]:
input_dict['track_index_to_predict']

In [ ]:
sample['batch_size']

In [ ]:
sample["input_dict"]["scenario_id"]

In [ ]:
sample["input_dict"]["center_objects_world"].shape

In [ ]:
#(num_center_objects, num_objects, num_timestamps, num_attrs)
sample["input_dict"]["obj_trajs"].shape

In [ ]:
obj_trajs = sample["input_dict"]["obj_trajs"]

In [ ]:
#(num_center_objects, num_future_timestamps, 4): [x, y, vx, vy] ground truth trajectories
sample["input_dict"]['center_gt_trajs'].shape

In [ ]:
sample["input_dict"]['center_gt_trajs_src'].shape

In [ ]:
obj_trajs[0][0]

In [ ]:
sample["input_dict"]["timestamps"]